    1- test function
    2- MC tests

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import numpy as np

# to delete this cell
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format='retina'
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# make plots look nice
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
plt.rc('axes', labelsize=10)
plt.rc('legend', fontsize=8)
mpl.rcParams['ps.useafm'] = True
mpl.rcParams['pdf.use14corefonts'] = True
mpl.rcParams['text.usetex'] = True

# 1- test function

In [ ]:
from GPPY.monte_carlo_test_functions import (f_1, f_2, f_3, f_4, f_5,
                                             f_6, f_7,
                                             f_8,
                                             exact_integral_f_1, exact_integral_f_2,
                                             exact_integral_f_3, exact_integral_f_4, 
                                             exact_integral_f_5, exact_integral_f_6,
                                             exact_integral_f_7,
                                             exact_integral_f_8,
                                             support_integrands)


In [ ]:
for d in range(2,8):
    print("for d={}".format(d))
    i_f = [{"f_"+str(i): globals()["exact_integral_f_{}".format(i)](d)} for i in range(1,9)]
    print("Exact integral functions=", i_f )

In [ ]:
r=1 #radius observation window
x = np.linspace(-r,r, 80)
X, Y = np.meshgrid(x, x)
points = np.array([X.ravel(), Y.ravel()]).T
z_f_1 = f_1(points)
z_f_2 = f_2(points)
z_f_3 = f_3(points)
z_f_4 = f_4(points)
z_f_5 = f_5(points)
z_f_6 = f_6(points)
z_f_7 = f_7(points)
z_f_8 = f_8(points)
fig = plt.figure(figsize=(14, 4))
ax = fig.add_subplot(2, 4, 1, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_1, c=z_f_1)
ax.set_title(r"$f_1$")
ax = fig.add_subplot(2, 4, 2, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_2, c=z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(2, 4, 3, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_3, c=z_f_3)
ax.set_title(r"$f_3$")
ax = fig.add_subplot(2, 4, 4, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_4, c=z_f_4)
ax.set_title(r"$f_4$")
ax = fig.add_subplot(2, 4, 5, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_5, c=z_f_5)
ax.set_title(r"$f_5$")
ax = fig.add_subplot(2, 4, 6, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_6, c=z_f_6)
ax.set_title(r"$f_6$")
ax = fig.add_subplot(2, 4, 7, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_7, c=z_f_7)
ax.set_title(r"$f_7$")
ax = fig.add_subplot(2, 4, 8, projection='3d')
ax.scatter3D(X.ravel(), Y.ravel(), z_f_8, c=z_f_8)
ax.set_title(r"$f_7$")
plt.show()

In [ ]:
r=1 #radius observation window
x = np.atleast_2d(np.linspace(-r,r, 140)).T
z_f_1 = f_1(x)
z_f_2 = f_2(x)
z_f_3 = f_3(x)
z_f_4 = f_4(x)
z_f_5 = f_5(x)
z_f_6 = f_6(x)
z_f_7 = f_7(x)
z_f_8 = f_8(x)

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(2, 4, 1)
ax.plot(x, z_f_1 )
ax.set_title(r"$f_1$")
ax = fig.add_subplot(2, 4, 2)
ax.plot(x, z_f_2)
ax.set_title(r"$f_2$")
ax = fig.add_subplot(2, 4, 3)
ax.plot(x,  z_f_3)
ax.set_title(r"$f_3$")
ax = fig.add_subplot(2, 4, 4)
ax.plot( x, z_f_4)
ax.set_title(r"$f_4$")
ax = fig.add_subplot(2, 4, 5)
ax.plot(x, z_f_5)
ax.set_title(r"$f_5$")
ax = fig.add_subplot(2, 4, 6)
ax.plot(x, z_f_6)
ax.set_title(r"$f_6$")
ax = fig.add_subplot(2, 4, 7)
ax.plot(x, z_f_7)
ax.set_title(r"$f_7$")
ax = fig.add_subplot(2, 4, 8)
ax.plot(x, z_f_8)
ax.set_title(r"$f_8$")
plt.show()

# 2- MC tests 

In [ ]:
from GPPY.monte_carlo_tests_setup import mc_results, plot_mc_results, dataframe_mse_results

In [ ]:
nb_point_list=[100, 500, 1000, 1500]

## d=2

In [ ]:
d=2
nb_sample=100
nb_function=7
support_window = support_integrands(d) 
estimators = ["MC", "MCR", "MCP", "MCDPP",
              #"MCKS_h0", "MCKSc_h0", 
              "RQMC", "MCCV"]
if __name__ == "__main__":
    mc_results_2d, nb_point_2d = mc_results(d, nb_point_list, nb_sample,
                                            nb_function, support_window, 
                                            estimators=estimators)

In [ ]:
dataframe_mse_results(d=d, mc_results=mc_results_2d, nb_function=nb_function,
                      nb_sample=nb_sample)

In [ ]:
dataframe_mse_results(d=d, mc_results=mc_results_2d, nb_function=nb_function,
                      nb_sample=nb_sample)

In [ ]:
plot_mc_results(d, mc_results_2d, nb_point_list, nb_sample, log_scale=False)

In [ ]:
from GPPY.monte_carlo_tests_setup import plot_mc_results

r = {
     'MCR':mc_results_2d['MCR'],
    'MC':mc_results_2d['MC'], 
     'MCP':mc_results_2d['MCP'],
     'MCDPP':mc_results_2d['MCDPP'],
     'MCPS':mc_results_2d['MCPS'],
     'RQMC': mc_results_2d['RQMC'],
     'MCCV':mc_results_2d['MCCV'],
}
plot_mc_results(d, r, nb_point_list, nb_sample, log_scale=False)

## d=3

In [ ]:
d=3
#N = [2**5, 2**6, 2**7, 2**8, 300, 400,  2**9, 700, 2**(10), 1300, 1500, 2**(11)]
#nb_point_list=[100, 300, 500, 1000, 1500, 2000]
nb_sample=100
nb_function=5
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_3d, nb_point_3d = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window, 
                                     correction=True)

In [ ]:
mse_3d_results = dataframe_mse_results(d=d, mc_results=mc_results_3d, nb_function=nb_function,
                      nb_sample=nb_sample)
mse_3d_results

In [ ]:
plot_mc_results(d, mc_results_3d, nb_point_list, nb_sample, save_fig=False)

In [ ]:
r = {
     'MCR':mc_results_3d['MCR'],
    'MC':mc_results_3d['MC'], 
     'MCP':mc_results_3d['MCP'],
     'MCDPP':mc_results_3d['MCDPP'],
     'RQMC': mc_results_3d['RQMC'],
     'MCCV':mc_results_3d['MCCV'],
}
plot_mc_results(d, r, nb_point_list, nb_sample, log_scale=False)

## d= 5

In [ ]:
d=5
#N = [2**5, 2**6, 2**7, 2**8, 300, 400,  2**9, 700, 2**(10), 1300, 1500, 2**(11)]
#nb_point_list=[100, 500, 1000, 1500]
nb_sample=100
nb_function=5
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_5d, nb_point_5d = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window, 
                                     correction=True)

In [ ]:
dataframe_mse_results(d=d, mc_results=mc_results_5d, nb_function=nb_function,
                      nb_sample=nb_sample)

In [ ]:
r = {
     'MCR':mc_results_5d['MCR'],
    'MC':mc_results_5d['MC'], 
     'MCP':mc_results_5d['MCP'],
     'MCDPP':mc_results_5d['MCDPP'],
     'RQMC': mc_results_5d['RQMC'],
     'MCCV':mc_results_5d['MCCV'],
}
plot_mc_results(d, r, nb_point_list, nb_sample, log_scale=False)

In [ ]:
plot_mc_results(d, r, nb_point, nb_sample, save_fig=False)

In [ ]:
mse_5d_results = dataframe_mse_results(mc_results_5d, nb_function)
mse_5d_results

In [ ]:
mse_5d_results = dataframe_mse_results(mc_results_2d, nb_function)

## d=6

In [ ]:
d=6
#N = [2**5, 2**6, 2**7, 2**8, 300, 400,  2**9, 700, 2**(10), 1300, 1500, 2**(11)]
#nb_point_list=[100, 500, 1000, 1500, 2000]
#nb_sample=40
#nb_function=6
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_6d, nb_point_6d = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window, 
                                     correction=False)

In [ ]:
plot_mc_results(d, mc_results_5d, nb_point, nb_sample, save_fig=False)

# d=8

In [ ]:
d=8
#N = [2**5, 2**6, 2**7, 2**8, 300, 400,  2**9, 700, 2**(10), 1300, 1500, 2**(11)]
nb_point_list=[100, 500, 1000]
nb_sample=50
nb_function=5
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_8d, nb_point_8d = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window, 
                                     correction=False)

In [ ]:
dataframe_mse_results(d=d, mc_results=mc_results_8d, nb_function=nb_function,
                      nb_sample=nb_sample)

In [ ]:
plot_mc_results(d, mc_results_2d, nb_point_list, nb_sample, save_fig="2d_mc_results.pdf")

# High dimension 

In [ ]:
from GPPY.monte_carlo_tests_setup import mc_fast_results

In [ ]:
d=10
#N = [2**5, 2**6, 2**7, 2**8, 300, 400,  2**9, 700, 2**(10), 1300, 1500, 2**(11)]
nb_point_list=[100, 500, 700]
nb_sample=50
nb_function=6
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_20d, nb_point = mc_fast_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window)

In [ ]:
import GPPY.monte_carlo_test_functions as mc_test_f

mc_test_f.exact_integral_f_1(2)

In [ ]:
import math

math.pi/4

# Test Pushed Sobol 

In [ ]:


d=5
#nb_point_list=[60]
nb_point_list=[ 100, 500, 1000]
estimtors = ["MC", "MCR", "MCP", "MCPS", "RQMC"]
nb_sample=100
nb_function=5
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_5d_, _ = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window,
                                           estimators=estimtors
                                           )

In [ ]:
plot_mc_results(d, mc_results_4d, nb_point_list, nb_sample, log_scale=False)

In [ ]:
from GPPY.spatial_windows import BallWindow
d=7
#nb_point_list=[60]
nb_point_list=[ 100, 500, 1000, 1500]
estimtors = ["MC", "MCR", "MCP", "MCPS", "RQMC"]
nb_sample=100
nb_function=2
# support 
r= min(np.diff(support_integrands(d).bounds, axis=1))/2
support_window = BallWindow(center=[0]*d, radius=r) 
if __name__ == "__main__":
    mc_results_4d_, _ = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window,
                                           estimators=estimtors
                                           )

In [ ]:
support_window.type()

In [ ]:
# for function with ball window 
d=7
#nb_point_list=[60]
nb_point_list=[ 100, 500, 1000, 1500]
estimtors = ["MC", "MCR", "MCP", "MCPS", "RQMC"]
nb_sample=100
nb_function=3
support_window =  
if __name__ == "__main__":
    mc_results_4d_, _ = mc_results(d, nb_point_list, nb_sample,
                                     nb_function, support_window,
                                           estimators=estimtors
                                           )

# trying to beat RQMC

In [ ]:
import multiprocessing

multiprocessing.cpu_count()

In [ ]:
nb_sample=50
nb_function=8
estimators = ["MC", "MCR",
              "MCP",
              "RQMC"]

In [ ]:
d=2
support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_2d, nb_point_2d = mc_results(d, nb_point_list, nb_sample,
                                            nb_function, support_window, 
                                            estimators=estimators, core_number=70)

In [ ]:
dataframe_mse_results(d=d, mc_results=mc_results_2d, nb_function=nb_function,
                      nb_sample=nb_sample)

In [ ]:
plot_mc_results(2, mc_results_2d, nb_point_list, nb_sample, log_scale=False, save_fig="mc_2_d.pdf", plot_dim=1)

In [ ]:
d=3
support_window = support_integrands(d) 

if __name__ == "__main__":
    mc_results_3d, _ = mc_results(d, nb_point_list, nb_sample,
                                    nb_function, support_window, 
                                    estimators=estimators,
                                    core_number=70)

In [ ]:
plot_mc_results(3, mc_results_3d, nb_point_list, nb_sample, log_scale=False, save_fig="mc_3_d.pdf", plot_dim=1)

In [ ]:
d=4
support_window = support_integrands(d) 

if __name__ == "__main__":
    mc_results_4d, _ = mc_results(d, nb_point_list, nb_sample,
                                    nb_function, support_window, 
                                    estimators=estimators,
                                    core_number=50,
                                    add_r_push=0.1)

In [ ]:
plot_mc_results(d, mc_results_4d, nb_point_list, nb_sample, log_scale=False, plot_dim=1)

In [ ]:
d=4
support_window = support_integrands(d) 

if __name__ == "__main__":
    mc_results_4d_, _ = mc_results(d, nb_point_list, nb_sample,
                                    nb_function, support_window, 
                                    estimators=estimators,
                                    core_number=50,
                                    add_r_push=0.2)

In [ ]:
plot_mc_results(4, mc_results_4d_, nb_point_list, nb_sample, log_scale=False, save_fig="mc_4_d.pdf", plot_dim=1)

In [ ]:
d=5

support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_7d, _ = mc_results(d, nb_point_list, 
                                            nb_sample,
                                    nb_function, support_window, 
                                    estimators=estimators,
                                    core_number=50,
                                    add_r_push=0.2)

In [ ]:
plot_mc_results(5, mc_results_7d, nb_point_list, nb_sample, log_scale=False, save_fig="mc_5_d.pdf", plot_dim=1)

In [ ]:
d=6

support_window = support_integrands(d) 
if __name__ == "__main__":
    mc_results_6d_, _ = mc_results(d, nb_point_list, 
                                    nb_sample,
                                    nb_function, support_window, 
                                    estimators=estimators,
                                    core_number=50,
                                    add_r_push=0.2)

In [ ]:
plot_mc_results(d, mc_results_2d, nb_point_list, nb_sample, log_scale=False)